## Homework 3 - Supervised Learning II - MDS Computational Linguistics

## T3 Helper Code


In [1]:
# all the necessary imports
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch import optim
import torchtext
from torchtext.data import Field, LabelField
from torchtext.data import TabularDataset
from torchtext.data import Iterator, BucketIterator

In [2]:
# set the seed
manual_seed = 572
torch.manual_seed(manual_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:

x = torch.rand((2,5,10))   # batch size 2 with length 10 and 5 dim embedding.

in_dim = 5
filters = 4

cnn1d = nn.Conv1d(in_dim,filters,kernel_size=3,padding=1) 
max_pool = nn.MaxPool1d(kernel_size=2)
ad_max_pool = nn.AdaptiveMaxPool1d(1)
activation = nn.ReLU()
x = cnn1d(x)
x = activation(x)
print(x)
print("Take the highest value in each window using max pool")
print(max_pool(x))
print(ad_max_pool(x))


tensor([[[0.0000, 0.2302, 0.4976, 0.1462, 0.2659, 0.1160, 0.2331, 0.2770,
          0.1295, 0.3005],
         [0.2625, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000],
         [0.3008, 0.0134, 0.0624, 0.2059, 0.1582, 0.1532, 0.0704, 0.0551,
          0.0224, 0.0000],
         [0.1708, 0.0017, 0.0000, 0.0516, 0.0199, 0.0000, 0.1053, 0.0412,
          0.0722, 0.0000]],

        [[0.0933, 0.2024, 0.3714, 0.2602, 0.1871, 0.1673, 0.0322, 0.5629,
          0.1314, 0.5456],
         [0.0278, 0.0589, 0.0000, 0.1606, 0.0000, 0.0000, 0.1000, 0.0000,
          0.0000, 0.0000],
         [0.0788, 0.1129, 0.0000, 0.2778, 0.0000, 0.1022, 0.2206, 0.0000,
          0.2809, 0.0000],
         [0.1701, 0.1184, 0.0053, 0.0000, 0.0000, 0.0000, 0.0847, 0.0000,
          0.1222, 0.0000]]], grad_fn=<ReluBackward0>)
Take the highest value in each window using max pool
tensor([[[0.2302, 0.4976, 0.2659, 0.2770, 0.3005],
         [0.2625, 0.0000, 0.0000, 0.0000, 0.0000],
        

In [ ]:

#useful for calculating size after padding and kernel
OutputSize = Length+ p*2 -(kernel-1)


Helper Code for loading

In [6]:
import torchtext
from torchtext.data import Field, LabelField
from torchtext.data import TabularDataset

# define the white space tokenizer to get tokens
def tokenize_en(tweet):
    """
    Tokenizes English tweet from a string into a list of strings (tokens)
    """
    return tweet.strip().split()

# define the TorchText's fields
TEXT = Field(sequential=True, tokenize=tokenize_en, lower=True)
LABEL = Field(sequential=False, unk_token = None)


train, val, test = TabularDataset.splits(
    path="./data/sentiment-twitter-2016-task4/", # the root directory where the data lies
    train='train.tsv', validation="dev.tsv", test="test.tsv", # file names
    format='tsv',
    skip_header=False, # if your tsv file has a header, make sure to pass this to ensure it doesn't get proceesed as data!
    fields=[('tweet', TEXT), ('label', LABEL)])

TEXT.build_vocab(train, min_freq=3) # builds vocabulary based on all the words that occur at least twice in the training set
LABEL.build_vocab(train)

train_iter, val_iter, test_iter = BucketIterator.splits(
 (train, val, test), # we pass in the datasets we want the iterator to draw data from
 batch_sizes=(4,64,64),
 sort_key=lambda x: len(x.tweet), 
 sort=True,
# A key to use for sorting examples in order to batch together examples with similar lengths and minimize padding. 
 sort_within_batch=True
)

VOCAB_SIZE = len(TEXT.vocab.stoi)

WORD_VEC_SIZE=300
# Note, the parameters to Embedding class below are:
# num_embeddings (int): size of the dictionary of embeddings
# embedding_dim (int): the size of each embedding vector
# For more details on Embedding class, see: https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/sparse.py

    


In [7]:
print(VOCAB_SIZE)

3330


Example Model

In [8]:

class ConvNet(nn.Module):
  
  def __init__(self, layer_num, filtersize, filters,nonlin, output_size, VOCAB_SIZE,  WORD_VEC_SIZE):  #feel free to add additional parameters
    super(ConvNet, self).__init__()
    self.embedding = nn.Embedding(VOCAB_SIZE, WORD_VEC_SIZE, sparse=True)
    #your code here
    self.embedding.weight.data.normal_(0.0,0.05)
    self.layers = nn.ModuleList()
    self.nonlin = nonlin
    for i in range(layer_num):
        if i == 0:
            self.layers.append(nn.Conv1d(WORD_VEC_SIZE, filters, filtersize, padding=(filtersize-1)//2))
        else:
            self.layers.append(nn.Conv1d(filters,filters,filtersize, padding=(filtersize-1)//2))
        self.layers.append(self.nonlin)    
    self.max_layer = nn.AdaptiveMaxPool1d(1)
    self.output = nn.Linear(filters, output_size)
    self.softmax = nn.LogSoftmax(dim=1) 

  def forward(self, x):
    x = self.embedding(x.permute(1,0)).permute(0,2,1)
    for layer in self.layers:
        x = layer(x)
    x = self.max_layer(x).squeeze(dim=-1)
    x =self.softmax(self.output(x))  # [N,H]
    
    return x

  

In [10]:

from sklearn.metrics import accuracy_score
def train(loader,model,criterion,optimizer,device):
    total_loss = 0.0
    # iterate throught the data loader
    num_sample = 0
    for batch in loader:
        # load the current batch
        batch_input = batch.tweet
        batch_output = batch.label
        
        batch_input = batch_input.to(device)
        batch_output = batch_output.to(device)
        # forward propagation
        # pass the data through the model
        model_outputs = model(batch_input)
        # compute the loss
        cur_loss = criterion(model_outputs, batch_output)
        total_loss += cur_loss.item()

        # backward propagation (compute the gradients and update the model)
        # clear the buffer
        optimizer.zero_grad()
        # compute the gradients
        cur_loss.backward()
        # update the weights
        optimizer.step()

        num_sample += batch_output.shape[0]
    return total_loss/num_sample

# evaluation logic based on classification accuracy
def evaluate(loader,model,criterion,device):
    all_pred=[]
    all_label = []
    with torch.no_grad(): # impacts the autograd engine and deactivate it. reduces memory usage and speeds up computation
        for batch in loader:
             # load the current batch
            batch_input = batch.tweet
            batch_output = batch.label

            batch_input = batch_input.to(device)
            # forward propagation
            # pass the data through the model
            model_outputs = model(batch_input)
            # identify the predicted class for each example in the batch
            probabilities, predicted = torch.max(model_outputs.cpu().data, 1)
            # put all the true labels and predictions to two lists
            all_pred.extend(predicted)
            all_label.extend(batch_output)
            
    accuracy = accuracy_score(all_label, all_pred)
    return accuracy


Example Random Search (if you have trouble working with Skorch)

In [11]:
import scipy.stats

LEARNING_RATE=.1
MAX_EPOCHS=10

def random_search(num_iter):
    results = []
    for i in range(num_iter):
        config = {
            #define hyperparameters here
            "layers": scipy.stats.randint.rvs(1,3),
            "filters": scipy.stats.randint.rvs(10,200)
            
        }
        
        print("new config")
        print(config)
        model = ConvNet(config["layers"],3,config["filters"],nn.ReLU(),output_size=3, VOCAB_SIZE=VOCAB_SIZE, WORD_VEC_SIZE=WORD_VEC_SIZE)
        model.to(device)
        criterion = nn.NLLLoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

    
        max_val = 0
        best_epoch = 0
        for epoch in range(MAX_EPOCHS):
        # train the model for one pass over the data
            train_loss = train(train_iter,model,criterion,optimizer,device)  
        # compute the training accuracy
            train_acc = evaluate(train_iter,model,criterion,device)
        # compute the validation accuracy
            val_acc = evaluate(val_iter,model,criterion,device)
            if val_acc > max_val:
                max_val = val_acc
                best_epoch = epoch+1
        # print the loss for every epoch
            print('Epoch [{}/{}], Loss: {:.4f}, Training Accuracy: {:.4f}, Validation Accuracy: {:.4f}'.format(epoch+1, MAX_EPOCHS, train_loss, train_acc, val_acc))
        results.append((max_val,best_epoch,config))
    return results

In [12]:
random_search(3)

new config
{'layers': 2, 'filters': 110}
Epoch [1/10], Loss: 0.2467, Training Accuracy: 0.5157, Validation Accuracy: 0.4217
Epoch [2/10], Loss: 0.2451, Training Accuracy: 0.5157, Validation Accuracy: 0.4217
Epoch [3/10], Loss: 0.2364, Training Accuracy: 0.5763, Validation Accuracy: 0.4712
Epoch [4/10], Loss: 0.2268, Training Accuracy: 0.6090, Validation Accuracy: 0.4742
Epoch [5/10], Loss: 0.2190, Training Accuracy: 0.6288, Validation Accuracy: 0.4747
Epoch [6/10], Loss: 0.2089, Training Accuracy: 0.6398, Validation Accuracy: 0.4627
Epoch [7/10], Loss: 0.1961, Training Accuracy: 0.6480, Validation Accuracy: 0.4547
Epoch [8/10], Loss: 0.1820, Training Accuracy: 0.6607, Validation Accuracy: 0.4432
Epoch [9/10], Loss: 0.1661, Training Accuracy: 0.6787, Validation Accuracy: 0.4487
Epoch [10/10], Loss: 0.1533, Training Accuracy: 0.7137, Validation Accuracy: 0.4207
new config
{'layers': 1, 'filters': 26}
Epoch [1/10], Loss: 0.2439, Training Accuracy: 0.5527, Validation Accuracy: 0.4687
Epoch

[(0.4747373686843422, 5, {'layers': 2, 'filters': 110}),
 (0.4932466233116558, 4, {'layers': 1, 'filters': 26}),
 (0.4882441220610305, 9, {'layers': 1, 'filters': 84})]